In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from sklearn.preprocessing import LabelEncoder


In [2]:
# data = pd.read_csv('https://drive.google.com/file/d/1cctWeqIBFhFpPxGGoDvu_rfwtVhQsXyr/view?usp=sharing') #Generated Dataset
# data = pd.read_csv('https://raw.githubusercontent.com/Ani-RudE/urop/main/RandomForest/sensor_raw.csv') #Original Dataset
data = pd.read_csv('sensor_raw2.csv') #Cleaning Dataset
window_size = 14

label_encoder = LabelEncoder()
data['Class'] = label_encoder.fit_transform(data['Class'])

In [30]:
def extract_features(data):
    features = []
    labels = []
    driver_ratings = {}
    for driver_id, driver_data in data.groupby('DriverID'):
        X = []
        y = []
        for _, task_data in driver_data.groupby('Class'):
            for i in range(0, len(task_data) - window_size):
                window = task_data.iloc[i:i + window_size]
                feature_vector = []
                feature_vector.append(window['AccX'].mean())
                feature_vector.append(window['AccY'].mean())
                feature_vector.append(window['AccZ'].mean())
                feature_vector.append(window['GyroX'].mean())
                feature_vector.append(window['GyroY'].mean())
                feature_vector.append(window['GyroZ'].mean())
                X.append(feature_vector)
                y.append(window['Class'].values[0])

        X = np.array(X)  # Convert the feature list to a NumPy array
        y = np.array(y)

        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42)
                                        
        # Create a simple ANN model with alternative activation functions
        model = Sequential()
        
     #    model.add(Dense(units=128, input_dim=6, activation='relu'))
     #    model.add(Dropout(0.2))
     #    model.add(Dense(units=64, activation='relu'))
     #    model.add(Dropout(0.2))
     #    model.add(Dense(units=32, activation='relu'))
     #    model.add(Dropout(0.2))
     #    model.add(Dense(units=16, activation='relu'))
     #    model.add(Dropout(0.2))
     #    model.add(Dense(units=60, activation='relu'))
     #    model.add(Dropout(0.2))

        model.add(Dense(units=256, input_dim=6, activation='relu', kernel_regularizer=l2(0.001)))
        model.add(Dropout(0.5))
        model.add(Dense(units=256, activation='relu', kernel_regularizer=l2(0.001)))
        model.add(Dropout(0.5))
        model.add(Dense(units=128, activation='relu', kernel_regularizer=l2(0.001)))
        model.add(Dropout(0.5))
        model.add(Dense(units=128, activation='relu', kernel_regularizer=l2(0.001)))
        model.add(Dropout(0.5))

        model.add(Dense(units=len(label_encoder.classes_), activation='softmax'))

        lr_schedule = ExponentialDecay(initial_learning_rate=0.01, decay_steps=1000, decay_rate=0.9)
        opt = Adam(learning_rate=lr_schedule)
        
        model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        #binary_crossentropy with activation='softmax'

        # Define Early Stopping callback
        early_stopping = EarlyStopping(
            monitor='val_loss',
            min_delta=0.0001,
            patience=20,
            verbose=1,
            mode='auto'
        )

        model.fit(X_train, y_train, validation_split=0.33, epochs=50, batch_size=100, validation_data=(X_test, y_test), callbacks=[early_stopping])  # Train the model

        # Evaluate the model on the test set
        accuracy = model.evaluate(X_test, y_test)[1]

        driver_ratings[driver_id] = accuracy

    return driver_ratings

In [31]:
driver_ratings = extract_features(data)

for driver_id, accuracy in driver_ratings.items():
    if accuracy > 0.9:
        driver_rating = "Excellent"
     #    print(driver_rating)
    elif accuracy > 0.8:
        driver_rating = "Good"
     #    print(driver_rating)
    elif accuracy > 0.7:
        driver_rating = "Average"
     #    print(driver_rating)
    else:      
        driver_rating = "Poor"
     #    print(driver_rating)

    print(f"{driver_id} Rating: {driver_rating}")

Epoch 1/50
3/3 [==============================] - 4s 335ms/step - loss: 1.9192 - accuracy: 0.3936 - val_loss: 1.1047 - val_accuracy: 0.6667
Epoch 2/50
3/3 [==============================] - 0s 48ms/step - loss: 1.2187 - accuracy: 0.6988 - val_loss: 0.9145 - val_accuracy: 0.8730
Epoch 3/50
3/3 [==============================] - 0s 44ms/step - loss: 1.0195 - accuracy: 0.8153 - val_loss: 0.8303 - val_accuracy: 0.8889
Epoch 4/50
3/3 [==============================] - 0s 44ms/step - loss: 0.8958 - accuracy: 0.8514 - val_loss: 0.7985 - val_accuracy: 0.8889
Epoch 5/50
3/3 [==============================] - 0s 44ms/step - loss: 0.9119 - accuracy: 0.8273 - val_loss: 0.7763 - val_accuracy: 0.8889
Epoch 6/50
3/3 [==============================] - 0s 46ms/step - loss: 0.8505 - accuracy: 0.8353 - val_loss: 0.7630 - val_accuracy: 0.8889
Epoch 7/50
3/3 [==============================] - 0s 48ms/step - loss: 0.8440 - accuracy: 0.8153 - val_loss: 0.7393 - val_accuracy: 0.8889
Epoch 8/50
3/3 [==========